In [1]:
#Generic
from collections import Counter
from collections import OrderedDict
import os
from os.path import join
import matplotlib.pyplot as plt
%matplotlib inline
import re
import numpy as np
import pandas as pd
from pprint import pprint
import pickle
import itertools
import datetime
from datetime import timedelta


#NLTK Stopword List
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('arabic')

#Gensim (LDA-Modelling)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaMulticore
from gensim.test.utils import datapath

#PDF-Reader
import pdfplumber

import pyLDAvis.gensim_models
import xml.etree.ElementTree as ETree

# plots
from bidi.algorithm import get_display
import arabic_reshaper
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
lda_model = LdaMulticore.load(r"C:\Users\kantg\Documents\Uni\nlp_arabic\saved_models\5_topics_pagewise_full")

with open('save_dict_pagewise_full.pkl', 'rb') as f:
    load_dat = pickle.load(f)
load_dat.keys()

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['data_df', 'pagewise_for_lda_chosen_tokens', 'id2word', 'corpus'])

In [3]:
df = load_dat['data_df']
df = df.sort_values(by='date')
df

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,page,file,date,cleaned_tokens,enemy_words_occurrences_counter,enemy_words_occurrences_sum
211,1 ـه1437/1/3 | لولأا ددعلا\nيملاعإ بتكم برقأ ع...,1,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[مضادا, فسيطروا, حاجزين, إمداد, وإدلب, حاول, أ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
213,3 ـه1437/1/3 | لولأا ددعلا\nّ\n)تيركت( لامشو )...,3,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[رافضي, سيارتهم, بعبوتين, ناسفتين, المنطقة, بع...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",2
214,ـه1437/1/3 | لولأا ددعلا 4\nّ\n)ةروبصلا( ةيرق ...,4,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[اللجاة, فجرت, الطرفين, اغتنم, المجاهدون, القل...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",9
215,5 ـه1437/1/3 | لولأا ددعلا\nّ\nّ\nيلشماقلا ةني...,5,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[القتلى, دائم, لعمليات, نوعية, تسفر, إعطابها, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",6
216,ـه1437/1/3 | لولأا ددعلا 6\nّ\n)ةريفسلا(و )راج...,6,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[البحرين, الخليج, جندي, الحمزة, بمناطق, وقطر, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",4
...,...,...,...,...,...,...,...
1782,7\nراــبخأ 315 ددعلا\nـه 1443 يناثلا عيبر 27 س...,7,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[ربيع, كانا, لتدمريها, الـkkp, مداهمة, منزله, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
1783,8\n315 ددعلا راــبخأ\nـه 1443 يناثلا عيبر 27 س...,8,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[آليتهم, مسلح, سويدان, ذيبان, مقديشو, الـkkp, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",7
1784,9\nتلااقم 315 ددعلا\nـه 1443 يناثلا عيبر 27 سي...,9,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[نعيم, الجنة, منكم, صدر, زاد, معرفته, والعلم, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
1785,10\n315 ددعلا ديهش ةصق\nـه 1443 يناثلا عيبر 27...,10,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[وقرية, وتعلم, قريته, مدة, للعمل, الكتب, وهن, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0


In [4]:
pages_per_months_df = df[df['enemy_words_occurrences_sum'] > 3]
pages_per_months_df = pages_per_months_df.sort_values(by='date')
pages_per_months_df

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,page,file,date,cleaned_tokens,enemy_words_occurrences_counter,enemy_words_occurrences_sum
214,ـه1437/1/3 | لولأا ددعلا 4\nّ\n)ةروبصلا( ةيرق ...,4,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[اللجاة, فجرت, الطرفين, اغتنم, المجاهدون, القل...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",9
215,5 ـه1437/1/3 | لولأا ددعلا\nّ\nّ\nيلشماقلا ةني...,5,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[القتلى, دائم, لعمليات, نوعية, تسفر, إعطابها, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",6
216,ـه1437/1/3 | لولأا ددعلا 6\nّ\n)ةريفسلا(و )راج...,6,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[البحرين, الخليج, جندي, الحمزة, بمناطق, وقطر, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",4
846,ـه1437/1/10 | يناثلا ددعلا 4\nاهفيرو بلح ةنيدم...,4,the-islamic-state-al-nabacc84_-newsletter-2.pdf,2015-10-23,"[بصاروخي, غراد, مباشرة, العناصر, لمنطقة, باجور...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",6
845,3 ـه1437/1/10 | يناثلا ددعلا\nّا ّا\n)ةشير وبل...,3,the-islamic-state-al-nabacc84_-newsletter-2.pdf,2015-10-23,"[الصفوية, وتصفيته, بهرز, الصفوية, استهدافه, نا...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",5
...,...,...,...,...,...,...,...
1749,صاخ\n4\n308 ددعلا راــبخأ\nـه 1443 لولأا عيبر ...,4,The-Islamic-State-al-Nabā-Newsletter-308.pdf,2021-09-03,"[صورة, معبد, بيشاور, بطلقات, مسدس, خوجياني, أب...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",6
1759,صاخ\n4\n312 ددعلا ريراقت\nـه 1443 يناثلا عيبر ...,4,The-Islamic-State-al-Nabā-Newsletter-312.pdf,2021-10-01,"[ربيع, وأحرق, تقريرا, مصورا, يوثق, اغتنمها, بت...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",7
1773,8\n314 ددعلا راــبخأ\nـه 1443 يناثلا عيبر 20 س...,8,The-Islamic-State-al-Nabā-Newsletter-314.pdf,2021-10-15,"[بنيران, باكستان, ربيع, بطلقات, بتوفيق, باكستا...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",4
1779,صاخ\n4\n315 ددعلا راــبخأ\nـه 1443 يناثلا عيبر...,4,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[حرارية, الرشاش, الثقيل, هجومهم, البيشمركة, يش...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",9


In [5]:
# part df monthwise
entries_monthwise = []
for year in range(2015, 2022):
    for month in range(1,13):
        df_append = df[df['date'].apply(lambda x: x.year == year and x.month == month)]
        df_append.reset_index(drop=True, inplace=True)
        if not df_append.empty: 
            entries_monthwise.append(df_append)

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
monthly_sum_of_occurrances = []
month = []
pages = []
for month_df in entries_monthwise:
    date = month_df['date'][0]
    date = date.replace(day=1)
    month.append(date)
    monthly_sum_of_occurrances.append(sum(month_df['enemy_words_occurrences_sum']))
    pages.append(month_df[month_df['enemy_words_occurrences_sum'] > 3].shape[0])
wrapper_dict = {'Time':month, 'Sum of outgroup-related words':monthly_sum_of_occurrances}
fig = px.bar(wrapper_dict, x='Time', y='Sum of outgroup-related words')
fig.update_layout(title='Occurrences of signal words over time')
fig.show()

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
wrapper_dict_pagewise = {'Time':month, 'Number of pages used as Training data': pages}
window_size = 3
i = 0
moving_averages = []
numbers = wrapper_dict_pagewise['Number of pages used as Training data']

while i < len(numbers) - window_size + 1:
    this_window = numbers[i : i + window_size]
    window_average = sum(this_window) / window_size
    moving_averages.append(window_average)
    i += 1
    #window_size += 1

moving_averages = [0,0,] + moving_averages
print(moving_averages)
print(len(moving_averages))


[0, 0, 17.333333333333332, 16.333333333333332, 10.666666666666666, 4.333333333333333, 5.666666666666667, 7.666666666666667, 7.666666666666667, 7.0, 6.666666666666667, 10.0, 9.666666666666666, 7.666666666666667, 4.0, 2.0, 3.3333333333333335, 5.333333333333333, 6.333333333333333, 7.0, 7.666666666666667, 9.0, 7.666666666666667, 6.333333333333333, 4.666666666666667, 3.6666666666666665, 3.3333333333333335, 4.333333333333333, 5.0, 4.666666666666667, 4.333333333333333, 4.333333333333333, 4.0, 3.0, 3.6666666666666665, 4.333333333333333, 4.0, 4.666666666666667, 4.333333333333333, 4.333333333333333, 4.333333333333333, 6.333333333333333, 7.333333333333333, 6.333333333333333, 3.3333333333333335, 2.0, 2.6666666666666665, 3.0, 4.333333333333333, 5.666666666666667, 8.0, 8.0, 6.333333333333333, 4.333333333333333, 3.6666666666666665, 3.3333333333333335, 3.6666666666666665, 4.0, 3.6666666666666665, 3.6666666666666665, 4.333333333333333, 4.666666666666667, 4.0, 4.0, 3.3333333333333335, 3.6666666666666665

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [23]:
fig = px.bar(wrapper_dict_pagewise, x='Time', y='Number of pages used as Training data', title='Number of Training pages per month', labels={'Number of pages used as Training data': 'Number of pages used <br> as Training data'})
fig.add_traces(go.Scatter(x=wrapper_dict_pagewise['Time'], y=moving_averages, mode = 'lines', name='Moving Average, window size: 3'))
fig.update_layout( font=dict(size=20))
fig.write_html("plotly_number_of_training_pages_per_month.html")
fig.show()

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [19]:
print(sum(wrapper_dict_pagewise['Number of pages used as Training data'][:37]))
sum(wrapper_dict_pagewise['Number of pages used as Training data'][37:])

237


c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



150

In [21]:
wrapper_dict_pagewise['Time'][37]

c:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



datetime.date(2018, 11, 1)

In [5]:
# new dfs with data by month and issue 
df_by_issue = pd.DataFrame(columns=['file', 'cleaned_tokens','date'])
df_by_month = pd.DataFrame(columns=['cleaned_tokens','date'])
for df in entries_monthwise:
    for name, date in zip(list(OrderedDict.fromkeys(df['file'])), list(OrderedDict.fromkeys(df['date']))): # ordered dict instead of set to keep order
        row = {'file': name , 'cleaned_tokens': list(itertools.chain(*df[df['date'] == date]['cleaned_tokens'].tolist())), 'date': date}
        df_by_issue = df_by_issue.append(row, ignore_index=True)
    row = {'cleaned_tokens': list(itertools.chain(*df['cleaned_tokens'].tolist())), 'date': datetime.datetime.strptime(df['date'][0].strftime('%Y-%m'), '%Y-%m')}
    df_by_month = df_by_month.append(row, ignore_index=True)
        

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
df_by_issue[:20]

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,cleaned_tokens,date
0,the-islamic-state-al-nabacc84_-newsletter-1.pdf,"[مضادا, فسيطروا, حاجزين, إمداد, وإدلب, حاول, أ...",2015-10-16
1,the-islamic-state-al-nabacc84_-newsletter-2.pdf,"[بصاروخي, غراد, مباشرة, العناصر, لمنطقة, باجور...",2015-10-23
2,the-islamic-state-al-nabacc84_-newsletter-3.pdf,"[الجنسية, جموعهم, المنطقة, يقل, الجنسية, والثا...",2015-10-30
3,the-islamic-state-al-nabacc84_-newsletter-4.pdf,"[حديثة, عامين, للنظام, طيران, سعيا, الحصار, يف...",2015-11-06
4,the-islamic-state-al-nabacc84_-newsletter-5.pdf,"[نتائجها, وكاسحة, ألغام, القاطع, بعملي, كلت, ا...",2015-11-13
5,the-islamic-state-al-nabacc84_-newsletter-6.pdf,"[احصل, المواد, أقرب, الفلوجة, ومدينة, شنه, إصد...",2015-11-20
6,the-islamic-state-al-nabacc84_-newsletter-7.pdf,"[احصل, المواد, أقرب, ضعفا, أضعاف, وقطر, اإلسال...",2015-11-27
7,the-islamic-state-al-nabacc84_-newsletter-8.pdf,"[أعقبتها, تسب, العملي, اإلسالمي, باغتنام, صيب,...",2015-12-04
8,the-islamic-state-al-nabacc84_-newsletter-9.pdf,"[بدء, موسم, نصارى, ورأس, تطبيق, الفتح, الفائت,...",2015-12-11
9,the-islamic-state-22al-nibacc84-newsletter-10.pdf,"[طيران, حالف, صليبي, بقصف, مسلمة, المئات, بدء,...",2015-12-18


In [7]:
df_by_issue['date'][:20]

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0     2015-10-16
1     2015-10-23
2     2015-10-30
3     2015-11-06
4     2015-11-13
5     2015-11-20
6     2015-11-27
7     2015-12-04
8     2015-12-11
9     2015-12-18
10    2015-12-25
11    2016-01-01
12    2016-01-08
13    2016-01-15
14    2016-01-22
15    2016-01-29
16    2016-02-05
17    2016-02-12
18    2016-02-19
19    2016-02-26
Name: date, dtype: object

In [8]:
df_by_month['date'][:20]

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    2015-10-01
1    2015-11-01
2    2015-12-01
3    2016-01-01
4    2016-02-01
5    2016-03-01
6    2016-04-01
7    2016-05-01
8    2016-06-01
9    2016-07-01
10   2016-08-01
11   2016-09-01
12   2016-10-01
13   2016-11-01
14   2016-12-01
15   2017-01-01
16   2017-02-01
17   2017-03-01
18   2017-04-01
19   2017-05-01
Name: date, dtype: datetime64[ns]

In [9]:
df_by_month

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,cleaned_tokens,date
0,"[مضادا, فسيطروا, حاجزين, إمداد, وإدلب, حاول, أ...",2015-10-01
1,"[حديثة, عامين, للنظام, طيران, سعيا, الحصار, يف...",2015-11-01
2,"[أعقبتها, تسب, العملي, اإلسالمي, باغتنام, صيب,...",2015-12-01
3,"[شديدة, االنفجار, فضال, حكومة, تصل, عقر, دارها...",2016-01-01
4,"[زوبع, ليؤدي, ربيع, باستهداف, مدفع, ملم, جبور,...",2016-02-01
...,...,...
67,"[بتوفيق, كمال, سريه, الجبوري, وصفهم, دعى, جبور...",2021-05-01
68,"[دمروا, تسعة, أبراج, محرم, ودمروا, انفجرت, بتو...",2021-07-01
69,"[نفوسهم, بالردة, شاء, ملة, قيل, هاديا, يهديهم,...",2021-08-01
70,"[الحدودية, مايل, نتائج, وإحداث, القوى, بقدرته,...",2021-09-01


In [10]:
df_by_issue
#df_by_issue[df_by_issue["cleaned_tokens"].str.len() == 0]

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,cleaned_tokens,date
0,the-islamic-state-al-nabacc84_-newsletter-1.pdf,"[مضادا, فسيطروا, حاجزين, إمداد, وإدلب, حاول, أ...",2015-10-16
1,the-islamic-state-al-nabacc84_-newsletter-2.pdf,"[بصاروخي, غراد, مباشرة, العناصر, لمنطقة, باجور...",2015-10-23
2,the-islamic-state-al-nabacc84_-newsletter-3.pdf,"[الجنسية, جموعهم, المنطقة, يقل, الجنسية, والثا...",2015-10-30
3,the-islamic-state-al-nabacc84_-newsletter-4.pdf,"[حديثة, عامين, للنظام, طيران, سعيا, الحصار, يف...",2015-11-06
4,the-islamic-state-al-nabacc84_-newsletter-5.pdf,"[نتائجها, وكاسحة, ألغام, القاطع, بعملي, كلت, ا...",2015-11-13
...,...,...,...
295,The-Islamic-State-al-Nabā-Newsletter-306.pdf,"[الرفق, حرم, عاقبته, غالبا, وصحبه, حرم, للدخول...",2021-08-20
296,The-Islamic-State-al-Nabā-Newsletter-308.pdf,"[الحدودية, مايل, نتائج, وإحداث, القوى, بقدرته,...",2021-09-03
297,The-Islamic-State-al-Nabā-Newsletter-312.pdf,"[وأحرقوا, شهد, وحواجز, للميليشيا, بعمليات, ثما...",2021-10-01
298,The-Islamic-State-al-Nabā-Newsletter-314.pdf,"[يصنع, الصدق, معنى, كذبا, أموره, صنع, فوق, قوم...",2021-10-15


In [11]:
# estimate topic distribution for the issues / monthwise

def estimate_topic_distr(series, lda_model):
    distr_df = pd.DataFrame(columns=['1', '2', '3', '4', '5'])
    corpus = [load_dat['id2word'].doc2bow(text) for text in series]
    for doc in corpus:
        topic_distr_vec = lda_model[doc][0]
        if len(topic_distr_vec) < 5: # in case a topic has a zero prevalence it is not returned. catch this scenario, i.e. topic_distr_vec == [(0, 0.1452677), (2, 0.4348741), (3, 0.1491024), (4, 0.26274395)]
            #print(topic_distr_vec)
            comp = []
            for i in range(len(topic_distr_vec)):
                comp.append(topic_distr_vec[i][0])
            zero_prev_topic_li = list(set([0,1,2,3,4]).difference(comp)) # get the number of the missing topic
            helper_dict = {}
            for zero_prev_topic in zero_prev_topic_li:
                #print(str(zero_prev_topic+1))
                helper_dict[str(zero_prev_topic+1)] = 0.0
            for num_prevalence_tupel in topic_distr_vec:
                helper_dict[str(num_prevalence_tupel[0]+1)] = num_prevalence_tupel[1]
            distr_df = distr_df.append(helper_dict, ignore_index=True)
        else:
            distr_df = distr_df.append({'1': topic_distr_vec[0][1], '2': topic_distr_vec[1][1], '3': topic_distr_vec[2][1], '4': topic_distr_vec[3][1], '5': topic_distr_vec[4][1]}, ignore_index=True)
    return distr_df

# concat the topic distributions:
df_by_issue = pd.concat([df_by_issue, estimate_topic_distr(df_by_issue['cleaned_tokens'], lda_model)], axis=1)
df_by_month = pd.concat([df_by_month, estimate_topic_distr(df_by_month['cleaned_tokens'], lda_model)], axis=1)

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
df_by_month.iloc[:,1:].to_csv("topics_by_month.csv")
df_by_issue.iloc[:,2:].to_csv("topics_by_issue.csv")

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [13]:
fig = go.Figure()
for i in range(1,6):
    fig.add_trace(go.Scatter(x=df_by_month['date'], y=df_by_month[str(i)],
                             mode='lines',
                             name='Topic ' + str(i)))
fig.update_layout(title='Topic Distribution over time for Tokens aggregated by month',
                  xaxis = dict(title='Time',
                            tick0 = df_by_month['date'].iloc[0],
                            tickvals = df_by_month['date'],
                            #ticktext = df_by_month['date']
                            ),
                  yaxis_title='Topic Proportion')
fig.write_html("plotly_topic_distr_monthwise.html")
fig.show()

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
fig = go.Figure()
for i in range(1,6):
    fig.add_trace(go.Scatter(x=df_by_issue['date'], y=df_by_issue[str(i)],
                             mode='lines',
                             name='Topic ' + str(i)))
fig.update_layout(title='Topic Distribution over time for Tokens aggregated by issue',
                  xaxis = dict(title='Time',
                            tick0 = df_by_month['date'].iloc[0],
                            tickvals = df_by_month['date'],
                            dtick = 1
                            #ticktext = df_by_month['date']
                            ),
                  yaxis_title='Topic Proportion')
fig.write_html("plotly_topic_distr_issuewise.html")
fig.show()

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

